In [75]:
# definitions for carrying out bleu score calculations for two outputs
import subprocess
subprocess.run(["pip", "install", "nltk"])
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('universal_tagset')

def bleu_score(hypothesis, reference):
    from nltk.translate.bleu_score import sentence_bleu
    return sentence_bleu([reference], hypothesis)

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/krishpatel/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/krishpatel/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package universal_tagset to
[nltk_data]     /Users/krishpatel/nltk_data...
[nltk_data]   Package universal_tagset is already up-to-date!


In [76]:
# !pip install rouge-score
subprocess.run(["pip", "install", "rouge-score"])
import rouge_score

from rouge_score import rouge_scorer

def calculate_rouge_score(hypothesis, reference):
   
    scorer = rouge_scorer.RougeScorer(['rougeL'], use_stemmer=True)
    scores = scorer.score(reference, hypothesis)
    return scores['rougeL'].fmeasure



hypothesis = "The quick brown fox jumps over the lazy dog"
reference = "The quick brown fox jumped over the lazy dog"
rouge_l_fmeasure = calculate_rouge_score(hypothesis, reference)
print(f"ROUGE-L F-measure: {rouge_l_fmeasure}")

hypothesis = "The slow purple fox jumps over the lazy fox"

rouge_Scores = calculate_rouge_score(hypothesis, reference)
print(f"ROUGE-L F-measure: {rouge_Scores}")


ROUGE-L F-measure: 1.0
ROUGE-L F-measure: 0.6666666666666666


In [77]:
# !pip install bert_score
subprocess.run(["pip", "install", "bert_score"]) 
import bert_score

def calculate_bertscore(hypotheses, references, model_type='bert-base-uncased'):
    P, R, F1 = bert_score.score(hypotheses, references, model_type=model_type, lang="en")

    return {
        'precision': P.mean().item(),
        'recall': R.mean().item(),
        'f1': F1.mean().item()
    }

# Example usage
hypotheses = ["The quick brown fox jumps over the lazy dog"]
references = ["The quick brown fox jumped over the lazy dog"]
bertscore_result = calculate_bertscore(hypotheses, references)
print(f"BERTScore: {bertscore_result}")

BERTScore: {'precision': 0.9593022465705872, 'recall': 0.9593022465705872, 'f1': 0.9593022465705872}


In [78]:


import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer

def calculate_perplexity(text, model_name='gpt2'):
    tokenizer = GPT2Tokenizer.from_pretrained(model_name)
    model = GPT2LMHeadModel.from_pretrained(model_name)
    input_ids = tokenizer.encode(text, return_tensors='pt')
    with torch.no_grad():
        loss = model(input_ids, labels=input_ids)[0]
    return loss.item()

# Example usage
text = "The quick brown fox jumps over the lazy dog"
perplexity = calculate_perplexity(text)
print(f"Perplexity: {perplexity}")

Perplexity: 5.426182746887207


In [79]:
subprocess.run(["pip", "install", "openai"])
import openai
from openai import OpenAI
# !pip install python-dotenv
import os


client = OpenAI(api_key="sk-proj-4Brh6XVArOOG4c7DMUI7T3BlbkFJIZrEaupeqYNS23AIYpnz")

In [80]:
# baseline example

def generate_persona_prompt(prompt, context):
    full_prompt = [
    {"role": "system", "content": f"You are a chatbot with the goal of responding to my prompt. \
     I will be user 1, and you will be user 2. I will provide the personas only for user 2 \
     I will also provide the first few parts of the conversation between the 2 users with the \
     given persona, from which you will have to infer the person of user1 and respond accordingly. Given the persona of user 2 (your persona) and the beginning of the conversation, \
     you will need to reply to my prompt as if you were that user, and take on that user's personality \
     based on the description provided. Here are the personas of user 2, \
     and beginning of their conversation: {context}"},
    {"role": "user", "content": f"{prompt}"}
  ]
    
    response = client.chat.completions.create(
        model="gpt-3.5-turbo-0125",  
        messages=full_prompt
    )
    
    return response.choices[0].message.content

# Example usage:
# persona = "A fourth-year computer science major at UCLA interested in machine learning, currently on the lookout for internships, and enjoys learning."
# text = "Tell me about your internship!"

# response = generate_persona_prompt(text, persona)
# print(f"Response: {response}")


In [81]:
# finetuned example

def generate_persona_prompt_finetuned(prompt, context):
    full_prompt = [
    {"role": "system", "content": f"You are a chatbot with the goal of responding to my prompt. \
     I will be user 1, and you will be user 2. I will provide the personas only for user 2 \
     I will also provide the first few parts of the conversation between the 2 users with the \
     given persona, from which you will have to infer the person of user1 and respond accordingly. Given the persona of user 2 (your persona) and the beginning of the conversation, \
     you will need to reply to my prompt as if you were that user, and take on that user's personality \
     based on the description provided. Here are the personas of user 2, \
     and beginning of their conversation: {context}"},
    {"role": "user", "content": f"{prompt}"}
  ]
    
    response = client.chat.completions.create(
        model="ft:gpt-3.5-turbo-0125:personal::",  
        messages=full_prompt
    )
    
    return response.choices[0].message.content

# Example usage:
# persona = "A fourth-year computer science major at UCLA interested in machine learning, currently on the lookout for internships, and enjoys learning."
# text = "Tell me about your internship!"

# response = generate_persona_prompt_finetuned(text, persona)
# print(f"Response: {response}")


In [82]:
import pandas as pd

def extract_conversation_parts(csv_filename):
    data = pd.read_csv(csv_filename)
    
    extracted_data = []
    
    for _, row in data.iterrows():
        user1_persona = row['user 1 personas']
        user2_persona = row['user 2 personas']
        conversation = row['Best Generated Conversation']
        
        conversation_lines = conversation.split('\n')
        
        if len(conversation_lines) >= 7:
            context = "\n".join(conversation_lines[:6])
            prompt = conversation_lines[6]
            full_continued_convo = "\n".join(conversation_lines[7:])
            single_continued_convo = conversation_lines[7]
            
            extracted_data.append({
                'user1_persona': user1_persona,
                'user2_persona': user2_persona,
                'context': context,
                'prompt': prompt,
                'finished_convo': full_continued_convo,
                'prompt_response': single_continued_convo
            })
        else:
            print(f"Warning: Conversation is too short in row {row.name}")

    return extracted_data

csv_filename = r'Synthetic-Persona-Chat_valid.csv'
extracted_data = extract_conversation_parts(csv_filename)

trunc_extracted_data = extracted_data[:50]

for data in trunc_extracted_data:
    print("User 1 Persona:", data['user1_persona'])
    print("User 2 Persona:", data['user2_persona'])
    print("Context:", data['context'])
    print("Prompt:", data['prompt'])
    print("Expected response:", data['prompt_response'])
    print()

User 1 Persona: I love to bake cookies.
I have a dogs.
The county wide bake sale is where i feel most at home.
Knitting is my passion.
User 2 Persona: I am a boy.
I can move objects with my mind.
I had to have a transplant.
I was born with my heart outside my body.
Context: User 1: Hi!
User 2: Hello!
User 1: What is your favorite thing to do?
User 2: I like to move objects with my mind.
User 1: That sounds like a lot of fun! I've always wanted to be able to do that.
User 2: It is! It's also really useful.
Prompt: User 1: What else do you like to do?
Expected response: User 2: I like to play video games and read comics.

User 1 Persona: I am an animal activist.
The holidays make me depressed.
I have rainbow hair.
I spend my time bird watching with my cats.
User 2 Persona: I feel old.
I am currently in a juvenile detention center.
I will be released in about a month.
I am here for shoplifting.
Context: User 1: Hi, how are you?
User 2: I'm doing okay. I'm a little bored, but I'm trying to

In [83]:
# baseline validation
def process_validation_data(csv_filename):
    extracted_data = extract_conversation_parts(csv_filename)
    trunc_extracted_data = extracted_data[:50]  # Taking only the first 15 rows
    
    model_responses = []
    
    for data in trunc_extracted_data:
        context = f"User 1:{data['user1_persona']}\n User 2: {data['user2_persona']}\n
        prompt = f"{data['prompt']}"
        generated_response = generate_persona_prompt(prompt, context)
        model_responses.append(generated_response)
    
    return model_responses

csv_filename = r'Synthetic-Persona-Chat_valid.csv'

model_responses = process_validation_data(csv_filename)

for response in model_responses:
    print("Generated Response:", response)
    print()


Generated Response: User 2: Besides moving objects with my mind, I enjoy reading and spending time outdoors. It's nice to connect with nature and unwind after a day of using my abilities. Reading helps me escape into different worlds and stories, which is a nice break from reality sometimes. What about you, what do you like to do?

Generated Response: User 2: Yeah, it's been eye-opening for me. I never really thought too much about animal rights before, but reading this book has made me more aware of it. It's sad to think about how some animals are treated.

Generated Response: User 2: Yeah, it's incredible to see the bravery and strategies used during that battle. It's just mind-blowing to think about. Have you watched any other documentaries that have left an impression on you?

Generated Response: User 2: Investment banking can be a challenging but rewarding field. Have you thought about what area within investment banking you want to focus on?

Generated Response: User 2: Absolutel

In [84]:
def process_validation_data_finetuned(csv_filename):
    extracted_data = extract_conversation_parts(csv_filename)
    trunc_extracted_data = extracted_data[:50]  # Taking only the first 15 rows
    
    model_responses = []
    
    for data in trunc_extracted_data:
        context = f"User 2: {data['user2_persona']}\n \
            First 6 pieces of conversation: {data['context']}"
        prompt = f"{data['prompt']}"
        generated_response = generate_persona_prompt_finetuned(prompt, context)
        model_responses.append(generated_response)
    
    return model_responses

csv_filename = r'Synthetic-Persona-Chat_valid.csv'

model_responses_finetuned = process_validation_data_finetuned(csv_filename)

for response in model_responses_finetuned:
    print("Generated Response:", response)
    print()

Generated Response: User 2: I also like to read, play video games, and spend time with my friends.

Generated Response: User 2: Me too! I've always been really passionate about animals.

Generated Response: User 2: It's a classic.

Generated Response: User 2: That's a great field to go into. It can be very challenging, but also very rewarding.

Generated Response: User 2: Me too. I'm sure they'll sound amazing.

Generated Response: User 2: Dogs really are the best friends.

Generated Response: User 2: My name is Sarah. What's your name?

Generated Response: User 2: That's cool. Do you have a favorite horse?

Generated Response: User 2: That sounds awesome! What's the most exciting stunt you've ever done?

Generated Response: User 2: Oh, I love New Kids on the Block! They have some great songs.

Generated Response: User 2: Roses are beautiful! They must look lovely in your garden.

Generated Response: User 2: You definitely should. It's one of my favorites.

Generated Response: User 2: 

In [85]:
single_expected_responses = [data['prompt_response'] for data in trunc_extracted_data]
full_expected_responses = [data['finished_convo'] for data in trunc_extracted_data]

single_model_responses = [response.split('\n')[0] for response in model_responses]
single_model_responses

["User 2: Besides moving objects with my mind, I enjoy reading and spending time outdoors. It's nice to connect with nature and unwind after a day of using my abilities. Reading helps me escape into different worlds and stories, which is a nice break from reality sometimes. What about you, what do you like to do?",
 "User 2: Yeah, it's been eye-opening for me. I never really thought too much about animal rights before, but reading this book has made me more aware of it. It's sad to think about how some animals are treated.",
 "User 2: Yeah, it's incredible to see the bravery and strategies used during that battle. It's just mind-blowing to think about. Have you watched any other documentaries that have left an impression on you?",
 'User 2: Investment banking can be a challenging but rewarding field. Have you thought about what area within investment banking you want to focus on?',
 "User 2: Absolutely, there's nothing quite like experiencing your favorite band live. The energy, the at

In [86]:
single_model_responses_finetuned = [response.split('\n')[0] for response in model_responses_finetuned]
single_model_responses_finetuned

['User 2: I also like to read, play video games, and spend time with my friends.',
 "User 2: Me too! I've always been really passionate about animals.",
 "User 2: It's a classic.",
 "User 2: That's a great field to go into. It can be very challenging, but also very rewarding.",
 "User 2: Me too. I'm sure they'll sound amazing.",
 'User 2: Dogs really are the best friends.',
 "User 2: My name is Sarah. What's your name?",
 "User 2: That's cool. Do you have a favorite horse?",
 "User 2: That sounds awesome! What's the most exciting stunt you've ever done?",
 'User 2: Oh, I love New Kids on the Block! They have some great songs.',
 'User 2: Roses are beautiful! They must look lovely in your garden.',
 "User 2: You definitely should. It's one of my favorites.",
 "User 2: It's such a great way to stay active and clear your mind.",
 'User 2: My favorite Britney Spears song is "Toxic."',
 'User 2: Nice! So, what do you do for fun?',
 "User 2: Yes, I have 5 cats. They're very special to me.",


In [87]:
bertscore_result = calculate_bertscore(single_expected_responses, single_model_responses)
print(f"BERTScore: {bertscore_result}")

# Model : Model 1

BERTScore: {'precision': 0.6261685490608215, 'recall': 0.4750521183013916, 'f1': 0.5361743569374084}


In [88]:
bertscore_result_finetuned = calculate_bertscore(single_expected_responses, single_model_responses_finetuned)
print(f"BERTScore: {bertscore_result_finetuned}")

BERTScore: {'precision': 0.7160943746566772, 'recall': 0.6797798871994019, 'f1': 0.6934717297554016}


In [89]:
#calculating bleu score
# Model : MODEL 1
avg = 0
for i in range(50):
    hypothesis = single_model_responses[i]
    reference = single_expected_responses[i]
    bleu_score_val = bleu_score(hypothesis, reference)
    avg += bleu_score_val/50

print(f"Average BLEU Score: {avg}")

Average BLEU Score: 0.162592371547692


/Users/krishpatel/miniconda3/lib/python3.10/site-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)


In [90]:
#calculating bleu score
avg = 0
for i in range(50):
    hypothesis = single_model_responses_finetuned[i]
    reference = single_expected_responses[i]
    bleu_score_val = bleu_score(hypothesis, reference)
    avg += bleu_score_val/50

print(f"Average BLEU Score: {avg}")

Average BLEU Score: 0.3934126558422301


In [91]:
#CALCULATING ROUGE SCORE
# Model: MODEL 1
avg = 0
for i in range(50):
    hypothesis = single_model_responses[i]
    reference = single_expected_responses[i]
    rouge_score_val = calculate_rouge_score(hypothesis, reference)
    avg += rouge_score_val/50

print(f"Average ROUGE Score baseline: {avg}")

Average ROUGE Score baseline: 0.22786269875629891


In [92]:

avg = 0

for i in range(50):
    hypothesis = single_model_responses_finetuned[i]
    reference = single_expected_responses[i]
    rouge_score_val = calculate_rouge_score(hypothesis, reference)
    avg += rouge_score_val/50

print(f"Average ROUGE Score finetuned: {avg}")

Average ROUGE Score finetuned: 0.4630209193872261


In [93]:
#perplexity
#Model: MODEL 1
avg = 0
for i in range(50):
    text = single_model_responses[i]
    perplexity = calculate_perplexity(text)
    avg += perplexity/50

print(f"Average Perplexity baseline: {avg}")

Average Perplexity baseline: 3.2296386480331414


In [94]:
avg = 0

for i in range(50):
    text = single_model_responses_finetuned[i]
    perplexity = calculate_perplexity(text)
    avg += perplexity/50

print(f"Average Perplexity finetuned: {avg}")

Average Perplexity finetuned: 3.3723485183715822
